In [6]:
import pandas as pd
import math
import numpy as np

# Read the data
def readFileWithoutOutliers():
    df = pd.read_csv("../data/data_train.csv")
    boolean_column = ['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']
    
    for column in boolean_column:
        df[column] = df[column].astype(bool)
    Q1 = df['fc'].quantile(0.25)
    Q3 = df['fc'].quantile(0.75)
    IQR = Q3 -Q1
    upper_bound = Q3 + 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR
    
    df_no_outliers = df[(df['fc'] > lower_bound) & (df['fc'] < upper_bound)]
    return df_no_outliers

def classifyData(data, bins1, labels1):
    classified_data = pd.cut(data, bins=bins1, labels=[str(label) for label in labels1])
    return classified_data
def changeBecameClassifyData(df):
    df["battery_power"] = classifyData(df["battery_power"], [500, 864, 1218, 1600, 1998], ['battery_power1', 'battery_power2', 'battery_power3', 'battery_power4'])
    df["clock_speed"] = classifyData(df["clock_speed"], [0.4, 0.7, 1.5, 2.2, 3], ["clock_speed1", "clock_speed2", "clock_speed3", "clock_speed4"])
    df["fc"] = classifyData(df["fc"], [-0.1, 1, 3, 7, 15], ["fc1", "fc2", "fc3", "fc4"])
    df["int_memory"] = classifyData(df["int_memory"], [1.9, 16, 32, 48, 64], ["int_memory1", "int_memory2", "int_memory3", "int_memory4"])
    df["m_dep"] = classifyData(df["m_dep"], [0, 0.2, 0.5, 0.8, 1], ["m_dep1", "m_dep2", "m_dep3", "m_dep4"])
    df["mobile_wt"] = classifyData(df["mobile_wt"], [79, 107.5, 139, 169, 200], ["mobile_wt1", "mobile_wt2", "mobile_wt3", "mobile_wt4"])
    df["n_cores"] = classifyData(df["n_cores"], [0.9, 2, 4, 7, 8], ["n_cores1", "n_cores2", "n_cores3", "n_cores4"])
    df["pc"] = classifyData(df["pc"], [-0.1, 5, 10, 15, 20], ["pc1", "pc2", "pc3", "pc4"])
    df["px_height"] = classifyData(df["px_height"], [-0.1, 273, 560, 946, 1960], ["px_height1", "px_height2", "px_height3", "px_height4"])
    df["px_width"] = classifyData(df["px_width"], [499.5, 878, 1247, 1623, 1998], ["px_width1", "px_width2", "px_width3", "px_width4"])
    df["sc_h"] = classifyData(df["sc_h"], [4.9, 9, 12, 16, 19], ["sc_h1", "sc_h2", "sc_h3", "sc_h4"])
    df["sc_w"] = classifyData(df["sc_w"], [-0.1, 2, 5, 9, 18], ["sc_w1", "sc_w2", "sc_w3", "sc_w4"])
    df["talk_time"] = classifyData(df["talk_time"], [1.9, 6, 11, 16, 20], ["talk_time1", "talk_time2", "talk_time3", "talk_time4"])
    return df 
def classify(namaKolom, value, df):
    if df[namaKolom].dtype != bool:
        Q1 = df[namaKolom].min()
        Q2 = df[namaKolom].quantile(0.25)
        Q3 = df[namaKolom].quantile(0.5)
        Q4 = df[namaKolom].quantile(0.75)
        Q5 = df[namaKolom].max()
        if(value >= Q1 and value <= Q2):
            return 1
        elif(value > Q2 and value <= Q3):
            return 2
        elif(value > Q3 and value <= Q4):
            return 3
        elif(value > Q4 and value <= Q5):
            return 4
    else:
        if value == True:
            return 1
        else:
            return 2

0
